## PROJECT


The data in this database has been pulled from SWAPI - A New Hope. As well as 'people', the API has data on starships. In Python, pull data on all available starships from the API. The "pilots" key contains URLs pointing to the characters who pilot the starship. Use these to replace 'pilots' with a list of ObjectIDs from our characters collection, then insert the starships into their own collection. 
Deliverables:
Your python file should: query the API, retrieve the starships (only keep useful fields). Transform the pilot list to contain a list of ObjectIds for the relative characters. Load the final starship documents into a new collection in MongoDB.
Extensions for project:
Improve your code by adding error handling, commenting, unit testing, etc. try to use look up function to bring those up from mongo db.


In [15]:
import requests
import json
import pymongo

#### 1. Pull data on all available starships from the API

In [16]:
# FUNCTION TO RETRIEVE DATA FROM API
def fetch_api_data(url):
    """Fetches api data from the provided URL and returns the results"""
    # we want to add all dictionairies to a list starship_dict
    starship_list = []
    try:    
        while url:
            # get url data using requests
            response = requests.get(url)
            data = response.json()

            # get the list of starships from the current page
            starship_list.extend(data["results"])

            # get the list of starships from the next page
            url = data["next"]

        # make the data look pretty
        # pretty_starship_list = json.dumps(starship_list, indent=4)
        return starship_list
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from API {e}")
        return None


In [17]:
starship = fetch_api_data("https://www.swapi.tech/api/starships")
pretty_starship = json.dumps(starship, indent=4)
print(pretty_starship)

[
    {
        "uid": "2",
        "name": "CR90 corvette",
        "url": "https://www.swapi.tech/api/starships/2"
    },
    {
        "uid": "3",
        "name": "Star Destroyer",
        "url": "https://www.swapi.tech/api/starships/3"
    },
    {
        "uid": "5",
        "name": "Sentinel-class landing craft",
        "url": "https://www.swapi.tech/api/starships/5"
    },
    {
        "uid": "9",
        "name": "Death Star",
        "url": "https://www.swapi.tech/api/starships/9"
    },
    {
        "uid": "11",
        "name": "Y-wing",
        "url": "https://www.swapi.tech/api/starships/11"
    },
    {
        "uid": "10",
        "name": "Millennium Falcon",
        "url": "https://www.swapi.tech/api/starships/10"
    },
    {
        "uid": "13",
        "name": "TIE Advanced x1",
        "url": "https://www.swapi.tech/api/starships/13"
    },
    {
        "uid": "15",
        "name": "Executor",
        "url": "https://www.swapi.tech/api/starships/15"
    },
    {
 

#### 2. Fetch data for each starship in a loop.

In [18]:
# QUERYING STARSHIP DETAILS
detailed_starships = []

for ship in starship:
    detail_url = ship['url']
    try:
        response = requests.get(detail_url)
        response.raise_for_status()
        starship_data = response.json()
        # append the starship details info inside detailed_starships
        detailed_starships.append(starship_data['result']['properties'])
        pretty_detailed_starships = json.dumps(detailed_starships, indent=4)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from API {e}")
# print(pretty_detailed_starships)
print(pretty_detailed_starships)
print(f"\nTotal detailed starships fetched: {len(detailed_starships)}")

[
    {
        "created": "2025-05-22T23:13:44.287Z",
        "edited": "2025-05-22T23:13:44.287Z",
        "consumables": "1 year",
        "name": "CR90 corvette",
        "cargo_capacity": "3000000",
        "passengers": "600",
        "max_atmosphering_speed": "950",
        "crew": "30-165",
        "length": "150",
        "model": "CR90 corvette",
        "cost_in_credits": "3500000",
        "manufacturer": "Corellian Engineering Corporation",
        "pilots": [],
        "MGLT": "60",
        "starship_class": "corvette",
        "hyperdrive_rating": "2.0",
        "films": [
            "https://www.swapi.tech/api/films/1",
            "https://www.swapi.tech/api/films/3",
            "https://www.swapi.tech/api/films/6"
        ],
        "url": "https://www.swapi.tech/api/starships/2"
    },
    {
        "created": "2025-05-22T23:13:44.287Z",
        "edited": "2025-05-22T23:13:44.287Z",
        "consumables": "2 years",
        "name": "Star Destroyer",
        "cargo_

#### 3. Trasnform starship data to show only necessary starship columns

In [19]:
starship_keys_to_keep = ["name",
                "manufacturer",
                "cargo_capacity",
                "length",
                "max_atmosphering_speed",
                "cost_in_credits",
                "crew",
                "passengers",
                "pilots"]

filtered_starship = []

for starship in detailed_starships:
    try:
        filtered = {key: starship[key] for key in starship_keys_to_keep}
        filtered_starship.append(filtered)
        pretty_filtered_starship = json.dumps(filtered_starship, indent=4)
    except KeyError as e:
        print(f"Missing key: {e}")
print(pretty_filtered_starship)



[
    {
        "name": "CR90 corvette",
        "manufacturer": "Corellian Engineering Corporation",
        "cargo_capacity": "3000000",
        "length": "150",
        "max_atmosphering_speed": "950",
        "cost_in_credits": "3500000",
        "crew": "30-165",
        "passengers": "600",
        "pilots": []
    },
    {
        "name": "Star Destroyer",
        "manufacturer": "Kuat Drive Yards",
        "cargo_capacity": "36000000",
        "length": "1,600",
        "max_atmosphering_speed": "975",
        "cost_in_credits": "150000000",
        "crew": "47,060",
        "passengers": "n/a",
        "pilots": []
    },
    {
        "name": "Sentinel-class landing craft",
        "manufacturer": "Sienar Fleet Systems, Cyngus Spaceworks",
        "cargo_capacity": "180000",
        "length": "38",
        "max_atmosphering_speed": "1000",
        "cost_in_credits": "240000",
        "crew": "5",
        "passengers": "75",
        "pilots": []
    },
    {
        "name": "D

#### 4. Retrieve pilot information (names) for each starship and add to existing data

In [20]:
for ship in filtered_starship:
    pilot_urls = ship['pilots']
    pilot_names = []
    if pilot_urls:
        for url in pilot_urls:
            try:
                response = requests.get(url)
                pilot_data = response.json()
                # append the pilot list to include information about pilots
                name =  pilot_data['result']['properties']['name']
                pilot_names.append({"name":name})
            except requests.exceptions.RequestException as e:
                print(f"Failed to fetch details for {detail_url}: {e}")
        ship['pilots'] = pilot_names
print(filtered_starship)
print(f"Total object IDs found: {len(pilot_names)}")


[{'name': 'CR90 corvette', 'manufacturer': 'Corellian Engineering Corporation', 'cargo_capacity': '3000000', 'length': '150', 'max_atmosphering_speed': '950', 'cost_in_credits': '3500000', 'crew': '30-165', 'passengers': '600', 'pilots': []}, {'name': 'Star Destroyer', 'manufacturer': 'Kuat Drive Yards', 'cargo_capacity': '36000000', 'length': '1,600', 'max_atmosphering_speed': '975', 'cost_in_credits': '150000000', 'crew': '47,060', 'passengers': 'n/a', 'pilots': []}, {'name': 'Sentinel-class landing craft', 'manufacturer': 'Sienar Fleet Systems, Cyngus Spaceworks', 'cargo_capacity': '180000', 'length': '38', 'max_atmosphering_speed': '1000', 'cost_in_credits': '240000', 'crew': '5', 'passengers': '75', 'pilots': []}, {'name': 'Death Star', 'manufacturer': 'Imperial Department of Military Research, Sienar Fleet Systems', 'cargo_capacity': '1000000000000', 'length': '120000', 'max_atmosphering_speed': 'n/a', 'cost_in_credits': '1000000000000', 'crew': '342,953', 'passengers': '843,342'

#### 5. Transform the pilot list to contain a list of ObjectIds for the relative characters and insert into their own collection

In [21]:
from pymongo import MongoClient

client = MongoClient()
db = client['starwars']

for starship in filtered_starship:
    pilot_entries = []
    for pilot in starship.get('pilots', []):
        pilot_name = pilot.get('name')
        if pilot_name:
            result = db.full_characters.find_one({"name": pilot_name}, {"_id": 1})
            if result:
                pilot_entries.append({'name': pilot_name, '_id': str(result['_id'])})
            else:
                pilot_entries.append({'name': pilot_name, '_id': None})  # not found
    starship['pilots'] = pilot_entries  # Replace with updated list of dicts


In [22]:
full_starship_data = json.dumps(filtered_starship, indent=4)
print(full_starship_data)

[
    {
        "name": "CR90 corvette",
        "manufacturer": "Corellian Engineering Corporation",
        "cargo_capacity": "3000000",
        "length": "150",
        "max_atmosphering_speed": "950",
        "cost_in_credits": "3500000",
        "crew": "30-165",
        "passengers": "600",
        "pilots": []
    },
    {
        "name": "Star Destroyer",
        "manufacturer": "Kuat Drive Yards",
        "cargo_capacity": "36000000",
        "length": "1,600",
        "max_atmosphering_speed": "975",
        "cost_in_credits": "150000000",
        "crew": "47,060",
        "passengers": "n/a",
        "pilots": []
    },
    {
        "name": "Sentinel-class landing craft",
        "manufacturer": "Sienar Fleet Systems, Cyngus Spaceworks",
        "cargo_capacity": "180000",
        "length": "38",
        "max_atmosphering_speed": "1000",
        "cost_in_credits": "240000",
        "crew": "5",
        "passengers": "75",
        "pilots": []
    },
    {
        "name": "D

#### 6. Load the final starship document into a new collection in MongoDB.

In [24]:
from pymongo import MongoClient

client = MongoClient()
db = client['starwars']

db.starships.delete_many({})

# Check the data type of filtered_starship
print(type(filtered_starship))

for i in range(0,36):
    db.starships.insert_one(filtered_starship[i])



<class 'list'>
